# Introduction to Numerical Modelling for Water Systems

In this lab you will write a simple model for how water infiltrates a porous soil column to replenish soil water storage. These exercises are intentionally simple, aiming to build up your conceptual understanding of how mathematical models can be developed and applied to environmental systems. The principles and concepts you will learn underlie most numerical models of water systems and are thus broadly applicable, rather than specific to the idealised system you will simulate in this lab.

The aims of this lab are to:
* Understand how a numerical model is constructed 
* Implement mathematical equations to describe how stored water volume changes through time
* Understand the importance of initial conditions in mathematical modelling
* Create and explain graphs/figures that visualise model behaviour
* Experiment with the model to understand its sensitivity to different parameters
* Understand the concepts of transient evolution and steady-state dynamic equilibrium
* Explore the concepts of model calibration and verification

### Python
The model is written in the Python computer programming language. But don't worry, you won't need any prior knowledge of computer programming. We have set up this lab so it will also serve as a basic introduction to programming. 

Learning to program is not the main aim but we want you to see what's "under the hood" of simple models to help you understand how they are built and what they can and cannot do! However, scientific computing is becoming more common place in research and consultancy, so it won't do you any harm to see it in action. Python is highly versatile, for example it can interface with GIS to automate workflows, or it is used in many data analysis and machine learning applications.

**To run a code cell, click in a cell, hold down shift, and press enter.** An asterisk in square brackets `In [*]:` will appear while the code is being executed, and this will change to a number `In [1]:` when the code is finished. *The order in which you execute code cells matters, they must be run in sequence. If the code fails, try restarting and executing all cells again from the top.* To restart the script click the circular arrow button or select `Kernel / Restart` from the menu bar at the top of the window.

Inside blocks of python code each line is a command which will be executed in sequence from top to bottom. Once a variable is created in a code cell it remains active and can continue to be used in further code cells below. There are comments indicated by lines that start with `#`. These lines are not computer code but rather provide commentary and information about what the code is doing to help you follow along. 

Python has many tools and we don't need to use all of them (it would take forever to load) so we have to tell python which modules we want to use:

In [ ]:
# import some modules that we will need for numerical calculations and for plotting
# we'll shorten their names using "as" so that we don't need to type much later on

# import modules for numerical calculations and for plotting
import numpy as np
import matplotlib.pyplot as plt

# tell python to allow plotting to occur within the page
%matplotlib inline

# Customise figure style to use font size 16
from matplotlib import rcParams
rcParams['font.size'] = 16

## 1.0 A smiple bathtub model

The simplest form of water balance in a soil column is akin to a bath tub that can fill up or drain depending on water being added by precipitation and soil infiltration ("from the tap") or lost by segregation towards aquifer recharge ("down the drain").

<img src="images/WaterBalance.png" alt="Drawing" style="width: 600px;"/>

The processes involved in our bathtub model are water flowing into the bathtub and out. These two processes together make up the total rate of change of stored water in the model, which is thus an open system. 

Following from the fundamental principle of mass conservation, we can write that the change in the volume ($V$) of water in the bathtub [m$^3$] over some time interval ($t$) [s] as determined by the volumetric rate of water coming in from the tap $Q_{in}$ [m$^3$/s] minus the rate of water being lost down the drain $Q_{out}$ [m$^3$/s]:

\begin{equation}
\dfrac{dV(t)}{dt} = Q_{in} - Q_{out} \tag{1} \label{eq:1}
\end{equation}

This is a so-called ordinary differential equation (ODE: it only contains a derivative of time, not space). Note that we will keep track of units as we go along to check for physical consistency. 

Equation \eqref{eq:1} describes the continuous evolution of volume over time. When translating this analytical model to a numerical model, we need to switch from a continuous representation of variables to a discrete one. Discretising an equation simply means that we divide up continuous variables such as the time $t$ into discrete instances of time $t^k$, where $k=0,1,2,...,N$ is a numbering index. 

Here, we are using the finite-difference method, which bears its name because it approximates the infinitesimal derivative operator, $d(\cdot)/dt$, with finite differences, 

\begin{equation}
    \dfrac{dV(t)}{dt} \approx \dfrac{V(t^k) - V(t^{k-1})}{t^k - t^{k-1}} \ . \tag{2} \label{eq:2}
\end{equation}

The discrete representation of \eqref{eq:1} then reads,

\begin{equation}
    {V^k - V^{k-1} \over \Delta t} = Q_{in} - Q_{out} \ , \tag{3} \label{eq:3}
\end{equation}

where we have simplified notation a little for convenience, such that $V^k = V(t^k)$ is the volume at discrete time $t^k$, and $\Delta t = t^k - t^{k-1}$ is the discrete time step.

We can rearrange \eqref{eq:3} to give an explicit description of the volume at time $t^k$, if we know the volume at a previous time $t^{k-1}$ in the past, along with the current fluxes $Q_{in}$, and $Q_{out}$,

\begin{equation}
    V^k = V^{k-1} + \left(Q_{in} - Q_{out}\right) \Delta t \ . \tag{4} \label{eq:4}
\end{equation}

This is called an explicit statement because we have only our *unknown variable* on the left-hand side, and all *known quantities* on the right-hand side. As we will see below, this model equation is quite straightforward to translate into computer code. 

We now need to define a procedural model or algorithm, i.e. the sequence of steps by which this mathematical description will be implemented as computer code. We can visualise how this is going to work using a flow diagram:

<img src="images/bathtub_flow_diagram.png" alt="Drawing" style="width: 400px;"/>

We will start with the simplest version of this model, where we assume both fluxes are constant. We start translating our algorithm ino Python code by defining our main variable, the volume $V$ at time $k=0$, along with two model parameters, which in our simple model are the two constant fluxes:

In [ ]:
V0   = 10. # initial volume of water in the bath tub [m3]
Qin  = 1.  # rate at which water is added to the bathtub from the tap [m3/s]
Qout = 1.  # rate at which water is drained from the bathtub down the drain [m3/s]

Next we define some numerical parameters to control the discrete representation of time in our simulation:

In [ ]:
t     = 0.   # initial time is zero [s]
tend  = 100. # time to stop the simulation [s]
dt    = 1.   # length of each time step [s]
steps = (int)((tend/dt)+1) # the number of timesteps it will take to reach the defined stopping time

`Python: (int)(.)` makes sure that the result of the operation in the attached brackets is an integer number (0, 1, 2, etc.) rather than a floating point number `(float)(.)` (0.0, 0.1, 0.11, etc.). In most situations, Python figures out on its own if a variable or parameter is `float` or `int`, for example by how you define it: `a = 1` will be interpreted as `int`, whereas `a = 1.` will be recognised as `float`. Just above we needed to make sure that the number of time steps is an integer, so we told Python to do just that.

We now create a couple of vector arrays to store the results of the model simulation as it evolves through time. The number of elements in the arrays will be the number of timesteps the model will take. A vector array is effectively a list of numbers, like a row of values in a spreadsheet. We create arrays initially containing all zero values using the function `np.zeros(steps)` from the numpy library. We create one array, `Time`, to store the discrete times $t^k$ and another, `Volume`, to store the corresponding volumes $V^k$ for all model steps:

In [ ]:
Time   = np.zeros(steps)  # array of zeros with as many entries as time steps
Volume = np.zeros(steps)  # array of zeros with as many entries as time steps

To illustrate what this has done, we will print one of the arrays to screen. You can print the value of any variable using `print(VariableName)`.

In [ ]:
print(Volume)

Alternatively we can print just the first value in the array, which has an index position of `k = 0` (as defined above),

In [ ]:
k = 0                # an index to point to a certain element in an array
print(Volume[k])     # print value at index position k

To run a simulation, we need to tell the computer to keep repeating the same set of calculations to update volume and time at every step until the stopping time is reached. To do that we will use a so-called `while` loop. We give the computer a logic condition to test, and it will carry on repeating calculations as long as that condition remains true. The condition we will use here is `t < tend`: while the simulation time is less than the stopping time, the same set of calculations will get repeated. We will update (increment) the time `t` and counting index `k` each time we pass through the loop. 

In [ ]:
# Set the initial volume in the bath tub and store it in the results vector
V         = V0
Volume[0] = V

while t < tend-dt/2:  # -dt/2 is introduced here to avoid overshooting the target stopping time by round-off errors
        
    # Calculate how much water is added from the tap
    V += Qin*dt
    
    # Calculate how much water is lost down the drain
    V -= Qout*dt
    
    # Increment time by adding the length of a time step
    t += dt
    
    # Increment counting index by adding one
    k += 1
    
    # Store updated volume and time in arrays
    Volume[k] = V
    Time[k]   = t

Note that all commands within the loop are indented by exactly one tab. This is not just to make the code look tidy but is necessary for Python to recognise which instructions belong inside the loop and should be repeated every time the loop is repeated.

`Python: a += b | -= b | *= b | /= b` is a shorthand for the operations,

`a = a+b | = a-b | = a*b | = a/b`

For example, above we used

`t += dt` 

as shorthand for 

`t = t + dt` .

## Experiment 1.1
Now that the model has run, we can plot the results. 

___QUESTION: What are your expectations for the volume of water in the bathtub based on the parameters we have chosen for the first simulation run?___

We use some commands from the `matplotlib.pyplot` (`plt` for short) library to create figures with plots. These should be fairly self-explanatory:

In [ ]:
# plot the results with a blue line
plt.plot(Time,Volume,'b-')
plt.xlabel('Time [s]')
plt.ylabel('Volume [m$^3$]');

Given that we set the rate of water flowing from the tap to be equal to the rate of water flowing down the drain, it is not surprising that the volume of water in the bath tub is constant through time at the initial value of 10 m$^3$. Thus, we consider the bath tub system to be in a state of **dynamic equilibrium**.

___QUESTION: What do you think will happen if we changed the rate of water flow from the tap or down the drain?___

To run further simulations more quickly, we are going to rewrite the algorithm above as a **function**. A function is basically a shortcut that will execute all commands within the function definition whenever the function is called. This allows to repeatedly run a sequence of code without the need to repeat all instructions each time. 

A function definition always starts with `def FunctionName(List of Arguments)`. You can give the function any name you find useful. You will need to state which variables and parameters you want to pass into the function every time you run it (in computer lingo these are called arguments). All other parameters not included in the list of arguments will be hard-coded into the function and can only be changed by messing with the function definition itself. A function definition always ends with a `return (List of Outputs)`, where you choose which updated variables or parameters should be returned to the user when calling the function.

___INSTRUCTION: find the missing block of code where we first wrote the model above to complete the while-loop within the function definition below!___

In [ ]:
def BathTubModel(V, Qin, Qout, tend):

    dt    = 1                    # the length of a time step
    steps = (int)((tend/dt)+1)   # the number of time steps required to reach stopping time
    
    Time   = np.zeros(steps)     # array of zeros with length steps
    Volume = np.zeros(steps)     # array of zeros with length steps
    
    # Set the initial V in the bath tub
    Volume[0] = V

    t = 0     # set the initial time to zero
    k = 0     # set the initial index to zero

    while t < tend-dt/2:
        
        # copy/paste the contents of the while-loop from above to here
        
        
    
    # return the model results when called
    return Time, Volume

Note again how all lines belonging to the function definition are indented by one tab. The code you have inserted inside the while-loop within the function now need to be indented two tabs from the base line to indicate that they belong _inside the loop, inside the function definition_, thus creating a neatly nested code structure.

Now we can **call** the function whenever we want to run the model and directly pass it the parameters to run the model. For example:

In [ ]:
# define model parameters
V0   = 10.  # initial volume [m3]
Qin  = 1.   # influx from the tap [m3/s]
Qout = 1.   # outflux down the drain [m3/s]
tend = 50   # stopping time [s]

# run the model with the prepared parameters as arguments
Time, Volume = BathTubModel(V0, Qin, Qout, tend)

# alternatively you can call the function by directly passing numbers as arguments
Time, Volume = BathTubModel(10.,1.,1.,50)

# plot the results with a blue line
plt.plot(Time,Volume,'b-')
plt.xlabel('Time [s]')
plt.ylabel('Volume [m$^3$]');

## Experiment 1.2
___INSTRUCTION: Explore how the model behaves for a range of different input parameters.___

Do so by modifying the initial volume of water in the bath, the flow rate from the tap or down the drain, or the stopping time in the code block below. Can you make the volume of water in the bath tub change through time?

In [ ]:
# try out different model parameters to develop an understanding of the model response
Time, Volume = BathTubModel( ... )

# plot the results using the appropriate lines of code from above
...

___QUESTION: What type of volume evolution histories are you able you produce? Are there any parameter choices where the model ceases to behave in a physically sensible way? What are some obvious limitations to our conceptualisation of the hydrological system as a bathtub?___

## Experiment 1.3
Now that you have experimented with the model, you will have noticed one of its limitations: if $Q_{in} < Q_{out}$ and the model is run for a sufficiently long time, the volume will reduce to values below zero, $V < 0$. Clearly, that is not a physically sensible outcome. 

**How can we improve the model?**

First, an improved model should satisfy the condition that outflow from the drain must stop when there is no more water in the tub: $Q_{out} \rightarrow 0$ for $V \rightarrow 0$. To keep things simple, we will make the assumption that flow down the drain depends linearly on how much water still is in the tub,
    
\begin{equation}
Q_{out} = R_{out} V \tag{5} \label{eq:5}
\end{equation}

where $R_{out}$ is a new model parameter, an outflow rate factor with units [1/s]. When multiplied with the volume $V$, the outflow rate $Q_{out}$ will again have units of [m$^3$/s]. Our updated (continuous) equation now reads,

\begin{equation}
{d V(t) \over dt} = Q_{in} - R_{out} V(t) \ , \tag{6} \label{eq:6}
\end{equation}

and the corresponding discretised equation becomes,

\begin{equation}
V^k = V^{k-1} + \left[Q_{in} - R_{out} V^{k-1}\right] \Delta t \tag{7} \label{eq:7}
\end{equation}

___INSTRUCTION: Create a new function that integrates this new model.___ 

Use the template prepared just below and complete the line where the volume is updated using eq. \eqref{eq:7}.

In [ ]:
def NewBathTubModel(V, Qin, Rout, tend):

    dt     = 1                   # the length of a time step
    steps  = (int)((tend/dt)+1)  # the number of timesteps that will take place during a model run

    Time   = np.zeros(steps)     # array of zeros with length steps
    Volume = np.zeros(steps)     # array of zeros with length steps
    
    # Set the initial V in the bath tub
    Volume[0] = V

    t = 0     # set the initial time to zero
    k = 0     # set the initial index to zero

    while t < tend-dt/2:
        
        # Calculate how much water is added from the tap and lost down the drain
        # translate the formula from eq. (7) to code and insert here

        
        
        # Increment time by adding the length of a time step
        t += dt

        # Increment counting index by adding one
        k += 1
        
        # Store updated volume and time in arrays
        Volume[k] = V
        Time[k]   = t   
    
    # return the model results when called
    return Time, Volume

___INSTRUCTION: Try out different parameter combinations to learn how the model responds.___

By now, you should have gotten the hang of it: copy and modify snippets of code from above to test the new model.

In [ ]:
# try out different input parameters to develop an understanding of the new model's sensitivity


# plot the results



Instead of the volume of water in the tub rising or sinking linearly as above, the new model now sees the volume gradually approach a final value that is again in **dynamic equilibrium**. This we call the **steady-state solution** of the model. 

___QUESTION: Can you figure out what model parameters the steady-state solution depends on?___ 

___INSTRUCTION: Investigate the model behaviour by running the model several times with different model parameters.___

To fully understand what is going on in your **sensitivity analysis**, only vary one parameter at the time while keeping all others at the same values. I have completed the first of three code blocks below to show you how it's done. The remaining two blocks are for you to complete, following the same pattern but testing the other parameters.

In [ ]:
# run the model with different inflow rates, but same initial volume and outflow rate factor
V0   = 15.
Rout = 0.1
Time_Qin05, Volume_Qin05 = NewBathTubModel(V0,0.5,Rout,100)
Time_Qin10, Volume_Qin10 = NewBathTubModel(V0,1.0,Rout,100)
Time_Qin15, Volume_Qin15 = NewBathTubModel(V0,1.5,Rout,100)

# plot all the results together
plt.plot(Time_Qin05,Volume_Qin05,'b-',Time_Qin10,Volume_Qin10,'r-',Time_Qin15,Volume_Qin15,'g-')
plt.xlabel('Time [s]')
plt.ylabel('Volume [m$^3$]')
plt.legend(['Qin = 0.5','Qin = 1.0','Qin = 1.5']);

In [ ]:
# run the model again with different outflow rate factors, but same initial volume and inflow rates
V0  = 15.
Qin = 1.
Time_Rout05, Volume_Rout05 = NewBathTubModel(V0,Qin,0.05,100)
Time_Rout10, Volume_Rout10 = 
# complete the sensitivity analysis of Rout following the pattern demonstrated above


# plot all the results together



In [ ]:
# run the model three times with different initial volumes, but same in/outflow parameters
Qin  = 1.
Rout = 0.1
Time_V05, Volume_V05 = NewBathTubModel( 5.,Qin,Rout,100)
# complete the sensitivity analysis of V0 as above

# plot all the results together



From these parameter tests we can see that the steady-state solution changes when we change either flux parameters, but does not change when we modify the initial volume only. This makes sense: a dynamic equilibrium is established when the influx of water from the tap exactly balances the outflux through the drain, no matter what volume of water is initially in the model tub.

Using this conceptual reasoning, we can directly calculate the steady-state volume, $V = V_\infty$, based on knowing the flux parameters only. The model equation (1) expresses that the total rate of volume change in the tub is $Q_{in} - Q_{out}$. The steady-state solution is reached when the volume no longer changes, that is, $dV/dt = 0$, and hence, $Q_{in} = Q_{out} = R_{out} V_\infty$. Solving this equation for $V_\infty$, we find that the steady-state solution is,

\begin{equation}
V_\infty = {Q_{in} \over R_{out}} . \tag{8} \label{eq:8}
\end{equation}

___INSTRUCTION: Double-check the units on the right-hand side of eq. \eqref{eq:8}; are they consistent with the units of volume on the left-hand side?___ 

Confirm that eq. \eqref{eq:8} indeed correctly predicts the steady-state solution by running the model again for different initial volumes as above, but now with one additional model run using the steady-state volume as initial value ($V_0 = V_\infty$). In the latter run the volume should not change at all over time! Let's see if this works...

In [ ]:
# calculate the steady-state solution based on our choice of flux parameters
Qin  = 1           # define inflow rate [m3/s]
Rout = 0.1         # define outflow rate factor [1/s]
Vss  = # <== fill in the formula from eq. (8) above!
print('steady-state volume =',Vss)  # print steady-state solution for your choice of parameters

# run the model three times with different initial volumes, but same flux parameters
Time_V05, Volume_V05 = NewBathTubModel( 5.,Qin,Rout,100)
Time_V15, Volume_V15 = NewBathTubModel(15.,Qin,Rout,100)
Time_V25, Volume_V25 = NewBathTubModel(25.,Qin,Rout,100)

# run the model once more, using the steady-state volume as initial state
Time_Vss, Volume_Vss = NewBathTubModel(Vss,Qin,Rout,100)

# plot all the results together
plt.plot(Time_V05,Volume_V05,'b-',Time_V15,Volume_V15,'r-',Time_V25,Volume_V25,'g-',Time_Vss,Volume_Vss,'k--')
plt.xlabel('Time [s]')
plt.ylabel('Volume [m$^3$]')
plt.legend(['V0 = 5','V0 = 15','V0 = 25','V0 = Vss']);

## Experiment 1.4
There is another useful piece of analysis we can perform on our bathtub model. Let's see what happens when comparing the following sets of flux parameters:

In [ ]:
# run the model again with different flux parameters
Time_1, Volume_1 = NewBathTubModel(20.,0.5,0.05,100)
Time_2, Volume_2 = NewBathTubModel(20.,1.0,0.10,100)
Time_3, Volume_3 = NewBathTubModel(20.,2.0,0.20,100)

# plot all the results together
plt.plot(Time_1,Volume_1,'b-',Time_2,Volume_2,'r-',Time_3,Volume_3,'g-')
plt.xlabel('Time [s]')
plt.ylabel('Volume [m$^3$]');

As you can see, these three model runs produce **transient evolution** curves that all trend towards the same **dynamic equilivrium** at $V_\infty = 10$, but they do so at different **equilibration rates**. The green curve gets close to $V_\infty$ by around 20, the red by about 40, and the blue just about by 80 seconds. 

___QUESTION: Can you explain by looking at the parameter choices used to produce these transient evolution paths why all three runs end on the same dynamic equilibrium?___

The fact that each of the above models approaches the steady state at a different rate means that the model must have a **characteristic time scale** $t_{eq}$ that predicts over what time span the bathtub volume approaches its dynamic equilibrium. 

It turns out that it is the outflow rate factor we introduced above that sets the characteristic equilibration time, $t_{eq} = 1/R_{out}$. For the parameter choices above, the predicted equilibration times are $1/0.05 = 20$ for blue, $1/0.1 = 10$ for red, and $1/0.2 = 5$ for green. Comparing to the plot above, we find that the model approaches the steady-state solution within $\sim 4 t_{eq}$. 

Mathematically, this is because the volume evolution of this model follows an exponential decay law, where $V(t) \sim \exp(-t/t_{eq})$. We won't go into that level of detail here, but this relationship can be proven by analytically solving the model ODE eq.(6). This relationship predicts that by a time of $4 t_{eq}$ the difference between $V(t)$ and $V_\infty$ will have reduced by a factor $\sim 10$ compared to the initial condition.

___INSTRUCTION: Based on what we have learned, calibrate the model to produce a given outcome.___ 

Identify the set of parameters that will lead to a model approaching a steady-state of $V_\infty = 21.5$ m$^3$ over a characteristic time of $t_{eq} = 12.5$ s!

In [ ]:
# set the flux parameters such that the model reaches Vss = 21.5 with teq = 12.5.
V0   = 10.;     # initial volume [m3]
Vss  = 21.5;    # given steady-state volume [m3]
teq  = 12.5;    # given equilibration time [s]
Qin  =    # <== fill in the appropriate formula to calibrate the influx rate given Vss and teq 
Rout =    # <== fill in the appropriate formula to calibrate the outflux rate factor given teq 
print('influx rate =',Qin,'\noutflux rate factor =',Rout)  # print calibrated parameter values

# run a simulation using your calibrated parameters
Time, Volume = NewBathTubModel(V0,Qin,Rout,100)

# plot all the results together to test if your calibration worked!
plt.plot(Time,Volume,'b-',[0,100],[Vss,Vss],'k--',[teq,teq],[V0,Vss],'k:',[5*teq,5*teq],[V0,Vss],'k:')
plt.xlabel('Time [s]')
plt.ylabel('Volume [m$^3$]')
plt.legend(['model','steady state','1x, 5x eq. time']);

Note that this exercise in **model calibraton** was reasonably simple; calibration of more complex models using observational or experimental data constraints will often be a rather involved procedure that rarely produces exact predictions. For example, just think about how often the weather forecast turns out to be correct...

## Experiment 1.5

An important part of model development is **model verification**, often also referred to as **benchmarking**. We need to make sure that the numerical discretisation used (in our example, splitting continuous physical time $t$ into discrete steps $\Delta t$) produces a good approximation to the true solution. 

To test that, we can perform a so-called **convergence test**, which involves running the model multiple times with different discrete step sizes $\Delta t$ and comparing the discrete, numerical solution to a known continuous or analytical one. If our model is implemented correctly, our approximate numerical solution should get closer to, or converge towards, the exact analytical solution as we reduce the discretisation step size. Note that in theory, if you reduce the discrete step to infinitesimally small size, you should recover the exact analytical solution!

___INSTRUCTION: To run a convergence test, modify the model function such that you can set the discrete time step when calling it.___ 

Below you find a mostly empty cell where you can copy in the contents of the `NewBathTubModel` function definition and modify it such that `dt` is no longer a **hard-coded parameter** within, but instead gets passed into the function as an **input argument**.

In [ ]:
def NewBathTubModelBenchm(V, Qin, Rout, tend, dt):

    # fill in the appropriate snippets of code to create a modified function 
    # where dt is passed as input argument
    
    # return the model results when called
    return Time, Volume

To benchmark the model code we also need to know the exact analytical solution to the problem, which can be found by analytically solving eq. (6). We won't go into the detail of how that is done here, but the exact analytical solution for the bathtub volume evolving through time is,

\begin{equation}
    V(t) = V_0 + (V_\infty - V_0) \left[1-\exp\left(-{t \over t_{eq}}\right)\right] \tag{9} \label{eq:9}
\end{equation}

On a side note, most models are much more complex than our bathtub example here; more often than not, there are no known analytical solutions to compare numerical solutions against. There are still ways to verify and benchmark a code but that is a more involved discussion we will have to leave for another day.

___INSTRUCTION: Run a set of simulations with the same input parameters but different time step sizes.___ 

This should work a treat if you completed `NewBathTubModelBenchm()` correctly above.

In [ ]:
# set the model parameters
V0   = 10.;
Qin  = 2;
Rout = 0.1;
Vss  = Qin/Rout;
teq  = 1/Rout;
tend = 60;

# run three simulations with different time step sizes
Time_dt4, Volume_dt4 = NewBathTubModelBenchm(V0,Qin,Rout,tend,4.)
Time_dt2, Volume_dt2 = NewBathTubModelBenchm(V0,Qin,Rout,tend,2.)
Time_dt1, Volume_dt1 = NewBathTubModelBenchm(V0,Qin,Rout,tend,1.)

# calculate the analytical solution coinciding with the time steps stored in Time_dt1
Volume_analyt = V0 + (Vss-V0) * (1-np.exp(-Time_dt1/teq))

# plot all the results together
plt.plot(Time_dt4,Volume_dt4,'b-',Time_dt2,Volume_dt2,'r-',Time_dt1,Volume_dt1,'g-',Time_dt1,Volume_analyt,'k--')
plt.xlabel('Time [s]')
plt.ylabel('Volume [m$^3$]')
plt.legend(['dt = 4','dt = 2','dt = 1','analytical']);

If it all worked out as planned, you should be able to easily convince yourself that your bathtub model indeed converges to the analytical solution as $\Delta t$ is reduced. 

As a final step, we want to be a bit more rigorous and actually quantify the numerical error of our benchmark runs. We define the numerical error as the root-mean-square norm of the difference between numerical and analytical solutions, $V_{num}$, and $V_{ana}$, respectively:

\begin{equation}
    E_{rms} = \sqrt{{\sum_k \left[V^k_{num} - V_{ana}(t^k)\right]^2 \over N}} \ , \tag{9} \label{eq:9}
\end{equation}

where $N$ is the number of time steps taken. 

___INSTRUCTION: Calculate the errors of the three benchmark runs above and compare them to the size of discrete time step taken in each.___

Numerical theory predicts that for the type of discretisation we use here the error should be reduced by the same factor by which the time step is reduced, a trend known as linear convergence. Complete the lines of code below to calculate the analytical solutions and RMS numerical errors for time step sizes of 4, 2, and 1 seconds.

In [ ]:
# Calculate the analytical solution on the same array dimensions as the three numerical solutions above
Volume_analyt_dt4 = ...
Volume_analyt_dt2 = ...
Volume_analyt_dt1 = ...

# Calculate the numerical error on the same array dimensions as the three numerical solutions above
E_dt4 = np.sqrt(np.sum((Volume_dt4 - Volume_analyt_dt4)**2)/np.size(Volume_dt4))
E_dt2 = ...
E_dt1 = ...

# plot all the results together
plt.plot(np.log10(4),np.log10(E_dt4),'bo',np.log10(2),np.log10(E_dt2),'ro',np.log10(1),np.log10(E_dt1),'go')
plt.plot(np.log10([4,2,1]),np.log10([E_dt4/1, E_dt4/2, E_dt4/4]),'k--')
plt.xlabel('log$_{10}$ Time step [s]')
plt.ylabel('log$_{10}$ Model error [m$^3$]')
plt.legend(['dt = 4','dt = 2','dt = 1','trend']);

If all has gone well, your numerical model will show the same convergence as the expected trend. Well done!

## Bonus Experiment

If you haven't seen enough of our bathtub model yet, here's one more thing you can try. We have just learned from our convergence test that a first-order scheme scheme gives linear convergence. It turns out it is not very difficult to improve model accuracy by using a second-order scheme that will give quadratic convergence. That means, for every factor we refine the time step, we get that factor squared reduction in numerical error! 

There's several options for second-order schemes, but here we will introduce the Crank-Nicolson scheme because of its simplicity. We first recall how we wrote the discrete model equation in (7) above,

\begin{equation}
V^k = V^{k-1} + \left[Q_{in} - R_{out} V^{k-1}\right] \Delta t \tag{10} \label{eq:10}
\end{equation}

Note that in this first-order scheme, we use the previous time step's volume, $V^{k-1}$, when calculating the volume-dependent outflow rate. Instead, in the Crank-Nicolson scheme, we use the volume at half a time step in the past, $V^{k-1/2} = \left(V^k + V^{k-1}\right)/2$. By modifying \eqref{eq:10} following this scheme we find,

\begin{equation}
V^k = V^{k-1} + \left[ Q_{in} - R_{out} \dfrac{V^k + V^{k-1}}{2} \right] \Delta t \ . \tag{11} \label{eq:11}
\end{equation}

While this is quite a simple expression, it is somewhat problematic that we now have the updated volume $V^k$ present on the right hand side of the equation, meaning that we would have to already know $V^k$ in order to calculate $V^k$. Clearly, that won't do! Instead, with a nifty move we can take the term containing $V^k$ over to the left hand side and write,

\begin{equation}
V^k \, \left(1 + {R_{out}\Delta t \over 2}\right) = V^{k-1} + \left[Q_{in} - {R_{out} V^{k-1} \over 2}\right] \Delta t \ . \tag{12} \label{eq:12}
\end{equation}

We can now divide both sides of the equation by $(1 + R_{out}\Delta t/2)$ and get,

\begin{equation}
V^k = {V^{k-1} + \left[Q_{in} - {R_{out} V^{k-1} \over 2} \right] \Delta t \over \left(1 + {R_{out}\Delta t \over 2} \right)} \ . \tag{13} \label{eq:13}
\end{equation}

Now we once again have only the unknown $V^k$ on the left hand side, and the known $V^{k-1}$ (solution of previous time step) on the right hand side of the equation; thus, we have again an explicit expression that is straightforward to translate into code.

___INSTRUCTION: Using code snippets from above along with \eqref{eq:13}, implement a second-order accurate version of your bathtub model!___

In [ ]:
def NewBathTubModelBenchm2nd(V, Qin, Rout, tend, dt):

    ...  # copy and modify appropriate lines of code from above to implement a second-order model version
    
    # return the model results when called
    return Time, Volume

___INSTRUCTION: Run the benchmark again using the same parameter choices and same time steps as above but employing your second-order model implementation now.___

In [ ]:
# run three simulations with different time step sizes
...

# plot all the results together
...

If all worked out as intended, it should be apparent just from looking at the plot that the second-order scheme is much more accurate. Practically no discrepancy should now be visible between numerical and analytical solution, even when using the same time step sizes as above. 

___INSTRUCTION: As above, quantify the numerical error of your second-order model implementation!___

In [ ]:
# Calculate the analytical solution on the same array dimensions as the three numerical solutions above
Volume_analyt_dt4 = ...
Volume_analyt_dt2 = ...
Volume_analyt_dt1 = ...

# Calculate the numerical error on the same array dimensions as the three numerical solutions above
E_dt4 = ...
E_dt2 = ...
E_dt1 = ...

# plot all the results together
plt.plot(np.log10(4),np.log10(E_dt4),'bo',np.log10(2),np.log10(E_dt2),'ro',np.log10(1),np.log10(E_dt1),'go')
plt.plot(np.log10([4,2,1]),np.log10([E_dt4/1, E_dt4/4, E_dt4/16]),'k--')
plt.xlabel('log$_{10}$ Time step [s]')
plt.ylabel('log$_{10}$ Model error [m$^3$]')
plt.legend(['dt = 4','dt = 2','dt = 1','trend']);

If you compare the RMS errors of the two time stepping schemes, it is easy to see quite how much better the second-order scheme performs. It doesn't only have more than one order of magnitude lower error at the largest time step (blue dot), but also reduces by a factor 4 in error for every factor 2 refinement of time step. 

The lesson we learn from this experiment is that numerical implementation matters. Even small changes can sometimes produce a much more accurate model. But in the end, the only way to find out how your model performs is to **rigorously benchmark your model for verification**!

**If you made it all the way to the end, well done! You're officially a scientific programmer now!**